# Imports 

In [20]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Get the data

In [35]:
os.chdir(r"C:\Users\Áron\Google Drive\UNI\Y4S2\Dissertation\Data")
CHOSEN_SHEETS = ["England", "Scotland"]

In [51]:
covid_data = pd.read_excel(r"summary_data_2018_2019.xlsx", skiprows = 3, sheet_name = CHOSEN_SHEETS)

In [54]:
covid_data

{'England':                                              Question       2018 Unnamed: 2  \
 0                                                 NaN  Full-time        NaN   
 1                                                 NaN    % agree  Responses   
 2                           The teaching on my course        NaN        NaN   
 3            1 - Staff are good at explaining things.         89     251764   
 4        2 - Staff have made the subject interesting.         82     251586   
 5       3 - The course is intellectually stimulating.         85     251659   
 6   4 - My course has challenged me to achieve my ...         81     251723   
 7                              Learning opportunities        NaN        NaN   
 8   5 - My course has provided me with opportuniti...         84     251730   
 9   6 - My course has provided me with opportuniti...         85     251561   
 10  7 - My course has provided me with opportuniti...         81     251519   
 11                          

In [53]:
covid_data.get("England")

,Question,2018,Unnamed: 2,Unnamed: 3,Unnamed: 4,2019,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,Full-time,NaN,Part-time,NaN,Full-time,NaN,Part-time,NaN
1,NaN,% agree,Responses,% agree,Responses,% agree,Responses,% agree,Responses
2,The teaching on my course,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1 - Staff are good at explaining things.,89,251764,86,19630,89,259777,86,21234
4,2 - Staff have made the subject interesting.,82,251586,82,19661,82,259593,80,21227
5,3 - The course is intellectually stimulating.,85,251659,89,20002,84,259704,88,21553
6,4 - My course has challenged me to achieve my ...,81,251723,84,19993,81,259749,84,21536
7,Learning opportunities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5 - My course has provided me with opportuniti...,84,251730,85,19980,84,259722,85,21538
9,6 - My course has provided me with opportuniti...,85,251561,84,19931,85,259638,83,21501
